# Plotting graphs in Datashader with edge bundling

[Datashader](http://datashader.readthedocs.org) provides general-purpose support for plotting very large collections of points, line segments, and gridded data, using a convenient API provided by [HoloViews](http://holoviews.org) and  an interactive user interface provided by [Bokeh](http://bokeh.pydata.org). While these facilities can be used to plot any sort of data, graph data (networks of nodes connected by edges) presents some special challenges.  At a minimum, the nodes and edges must be mapped into specific locations and shapes in a 2D plane before they can be visualized, which is an [open-ended problem involving a complex set of tradeoffs and complications](http://www.hiveplot.com).

Here, we will assume that the nodes have already been laid out in 2D, whether because they have such structure inherently (such as connections between geographic locations), or an arbitrary random or geometric layout has been selected manually, or the layout has been determined by an external tool (such as a force-directed algorithm laying out similarly connected nodes in similar locations).

First, we'll import the packages we are using and set up some defaults.

In [ ]:
import math

import numpy as np
import pandas as pd
import holoviews as hv
import fastparquet as fp

from colorcet import fire
from datashader.bundling import directly_connect_edges, hammer_bundle

from holoviews.operation.datashader import datashade, dynspread
from holoviews.operation import decimate

from dask.distributed import Client
client = Client()

hv.notebook_extension('bokeh','matplotlib')

decimate.max_samples=20000
dynspread.threshold=0.01
datashade.cmap=fire[40:]
sz = dict(width=150,height=150)

%opts RGB [xaxis=None yaxis=None show_grid=False bgcolor="black"]

## Random graph

As a first example, let's generate a random graph, with 5000 points normally distributed around the origin and 20000 random connections between them:

In [ ]:
np.random.seed(0)

nodes = hv.Points(pd.DataFrame(np.random.randn(5000, 2), columns=['x', 'y']))
nodes.data.tail()

In [ ]:
edges = hv.Curve(pd.DataFrame(np.random.randint(len(nodes.data), size=(20000, 2)),
                              columns=['source', 'target']))
edges.data.tail()

Here you can see that the nodes list is a columnar dataframe with an index value and an (x,y) location for every node.  The edges list is a columnar dataframe listing the index of the source and target in the nodes dataframe.  

We can now visualize this data using some graph-specific helper functions in Datashader, with results packaged as HoloViews objects that display using Bokeh:

In [ ]:
%%opts RGB [width=400 height=400]

%time direct  = hv.Curve(directly_connect_edges(nodes.data, edges.data))
%time bundled = hv.Curve(hammer_bundle(nodes.data, edges.data))

(datashade(direct)  * dynspread(datashade(nodes,cmap=["cyan"]),threshold=0.01)).relabel("Directly connected") + \
(datashade(bundled) * dynspread(datashade(nodes,cmap=["cyan"]),threshold=0.01)).relabel("Bundled")

In these plots, you can select the Bokeh "wheel zoom" from the tool palette and zoom or pan on the plot.  On a static site like anaconda.org the image will be rescaled blockily as you zoom in, not revealing any new data, but with a live server it will be dynamically re-rendered to show more detailed structure each time. Here the nodes are shown as cyan-colored dots, and the connections are lines that show up as dark red for a single line and gradually brighter colors as more lines are overlaid.  

The "Directly Connected" plot shows a single line segment per edge, which is very quick (scaling in computational complexity with the number of edges) but has so much overlap that it is difficult to see the underlying structure.

Conversely, if we render the edges as curves by using an edge bundling algorithm (a variant of [Hurter, Ersoy, & Telea, ECV-2012](http://www.cs.rug.nl/~alext/PAPERS/EuroVis12/kdeeb.pdf)), we can more easily see the structure of groups of connections, but here there is very little structure to see, given the overall randomness.  Other graphs will have more interesting structure, as shown below.

## Bundling example using a star graph

As a simple example with clear structure, let's put one node at the center, and select some random points on a circle around it that connect to the central node (a star graph topology):

<!-- def circle(r,n): return [(math.cos(2*math.pi/n*x)*r,math.sin(2*math.pi/n*x)*r) for x in range(0,n)] -->

In [ ]:
n = 75
np.random.seed(0)
x = np.random.random(n)

nodes = hv.Points(pd.DataFrame(np.stack((np.cos(2*math.pi*x),np.sin(2*math.pi*x))).T, columns=['x', 'y']))
nodes.data.iloc[0] = (0.0,0.0)
edges = hv.Curve(pd.DataFrame(list(zip((range(1,n+1)),[0]*n)),columns=['source', 'target']))

In [ ]:
%%opts RGB [width=400 height=400]

direct  = hv.Curve(directly_connect_edges(nodes.data, edges.data))
bundled = hv.Curve(hammer_bundle(nodes.data, edges.data))

(datashade(direct)  * dynspread(datashade(nodes,cmap=["cyan"]))).relabel("Directly connected") + \
(datashade(bundled) * dynspread(datashade(nodes,cmap=["cyan"]))).relabel("Bundled")

Here you can see the bundling algorithm forms groups of nearby connnections, which helps make the structure at a particular scale clear.  The scale of this structure, i.e., how much bundling is done, is determined by an effective "bandwidth", which is a combination of an `initial_bandwidth` parameter and a `decay` time constant for annealing this bandwidth over time:

In [ ]:
%time  grid = hv.GridSpace({(bw, decay): hv.Curve(hammer_bundle(nodes.data, edges.data, iterations=5, \
                                                         decay=decay, initial_bandwidth=bw)) \
                              for decay in [0.1, 0.25, 0.5, 0.9] for bw in [0.1, 0.2, 0.5, 1]}, \
                           kdims=['Initial bandwidth', 'Decay'])

In [ ]:
%%opts RGB [width=150 height=150] {+framewise +axiswise}
datashade(grid, **sz).map(lambda edges_ds: edges_ds * nodes, hv.DynamicMap)

Starting from the bottom left and moving diagonally to the upper right, the scale of the bundling increases along a diagonal to the upper right, with higher initial bandwidth and higher decay time constants leading to larger-scale bundling.  For the largest decay time constant, the algorithm has failed to converge for large initial bandwidths (the bw 0.5 and 1.0 plots on the top row), because the algorithm stops at a specified maximum `iterations`, rather than reaching a fully organized state.

Of course, even when the algorithm does converge, larger amounts of bundling can magnify small amounts of clumping over large scales, which may or may not be relevant to the questions being asked of this data, so it is important to set these parameters appropriately for the types of structures of interest.

<!--
max_iterations=10
hmap = hv.HoloMap({(it, bw, decay): hv.Curve(hammer_bundle(nodes.data, edges.data,
                         decay=decay, initial_bandwidth=bw, iterations=it))
                     for decay in [0.1, 0.25, 0.5, 1, 2] 
                     for bw in [0.1, 0.2, 0.5, 1] 
                     for it in range(max_iterations)},
                   kdims=['Iteration', 'Initial bandwidth', 'Decay'])
    
nodes_ds = datashade(nodes,cmap=["cyan"])
datashade(hmap.grid(['Initial bandwidth', 'Decay']), **sz).map(lambda e_ds: e_ds * nodes, hv.DynamicMap)
-->

## Using graphs from NetworkX

The above examples constructed networks by hand.  A convenient way to get access to a large number of [graph types](https://networkx.github.io/documentation/stable/reference/generators.html) is the separate [NetworkX](https://networkx.readthedocs.io) package.  Here, we will select several standard graph structures, lay them each out in the same fixed circular shape using NetworkX, and then show how they will appear without bundling, with moderate levels of bundling, and with high amounts of bundling.

In [ ]:
import networkx as nx

def nx_layout(graph):
    layout = nx.circular_layout(graph)
    data = [[node, *layout[node]] for node in graph.nodes]

    nodes_df = pd.DataFrame(data, columns=['id', 'x', 'y'])
    nodes_df.set_index('id', inplace=True)
    nodes = hv.Points(nodes_df)

    edges = hv.Points(pd.DataFrame(list(graph.edges), columns=['source', 'target']))
    return nodes, edges

def nx_plot(graph):
    print(graph.name, len(graph.edges))
    nodes, edges = nx_layout(graph)
    
    direct = hv.Curve(directly_connect_edges(nodes.data, edges.data))
    bundled_bw005 = hv.Curve(hammer_bundle(nodes.data, edges.data))
    bundled_bw030 = hv.Curve(hammer_bundle(nodes.data, edges.data, initial_bandwidth=0.30))

    return (datashade(direct, **sz)        * nodes).relabel(graph.name) + \
           (datashade(bundled_bw005, **sz) * nodes).relabel("Bundled bw=0.05") + \
           (datashade(bundled_bw030, **sz) * nodes).relabel("Bundled bw=0.30")

In [ ]:
%%time
%%opts RGB [width=200 height=200] {+framewise}

n=100

hv.Layout([nx_plot(g) for g in
           [nx.complete_graph(n), nx.lollipop_graph(n, 5),     nx.barbell_graph(n,2),
            nx.ladder_graph(n),   nx.circular_ladder_graph(n), nx.star_graph(n),
            nx.cycle_graph(n)]]).cols(3)

As you can see, both bundled and unbundled representations reflect important aspects of the graph structure, but the bundling results do depend on the parameters chosen.  Bundling is also very computationally expensive; nearly all of the time taken to render these plots is for the bundling step.

Note that the `star_graph` example above differs from the one in the previous section, in that all nodes here connect to a node on the outer circle instead of one in the center, which shows clearly how the layout can affect the resulting visualization.

## UK research collaboration example

The above examples use artificial datasets, but each real-world dataset will have its own specific properties. Here, let's look at one example dataset of 600,000 collaborations between 15000 UK research institutions, previously laid out using a force-directed algorithm by [Ian Calvert](https://www.digital-science.com/people/ian-calvert):

In [ ]:
r_nodes_file = 'data/calvert_uk_research2017_nodes.snappy.parq'
r_edges_file = 'data/calvert_uk_research2017_edges.snappy.parq'

r_nodes = hv.Points(fp.ParquetFile(r_nodes_file).to_pandas(index='id'), label="Nodes")
r_edges = hv.Curve( fp.ParquetFile(r_edges_file).to_pandas(index='id'), label="Edges")
len(r_nodes),len(r_edges)

We can render each collaboration as a single-line direct connection, but the result is a dense tangle:

In [ ]:
%%opts RGB [tools=["hover"] width=400 height=400] 

%time r_direct = hv.Curve(directly_connect_edges(r_nodes.data, r_edges.data),label="Direct")

dynspread(datashade(r_nodes,cmap=["cyan"])) + \
datashade(r_direct)

Detailed substructure of this graph becomes visible after bundling, which takes several minutes even using multiple cores with Dask:

In [ ]:
%time r_bundled = hv.Curve(hammer_bundle(r_nodes.data, r_edges.data),label="Bundled")

In [ ]:
%%opts RGB [tools=["hover"] width=400 height=400] 

dynspread(datashade(r_nodes,cmap=["cyan"])) + datashade(r_bundled)

Zooming into these plots reveals interesting patterns, but immediately one then wants to ask what the various groupings of nodes might represent. With a small number of nodes or a small number of categories one could color-code the dots (using datashader's categorical color coding support), but here we just have thousands of indistinguishable dots. Instead, let's use hover information so the viewer can at least see the identity of each node on inspection.  

To do that, we'll first need to pull in something useful to hover, so let's load the names of each institution in the researcher list and merge that with our existing layout data:

In [ ]:
node_names = pd.read_csv("data/calvert_uk_research2017_nodes.csv", index_col="node_id", usecols=["node_id","name"])
node_names = node_names.rename(columns={"name": "Institution"})
node_names

r_nodes_named = pd.merge(r_nodes.data, node_names, left_index=True, right_index=True)
r_nodes_named.tail()

We can now overlay a set of points on top of the datashaded edges, which will provide hover information for each node.  Here, the entire set of 15000 nodes would be reasonably feasible to plot, but to show how to work with larger datasets we wrap the `hv.Points()` call with `decimate` so that only a finite subset of the points will be shown at any one time. If a node of interest is not visible in a particular zoom, then you can simply zoom in on that region; at some point the number of visible points will be below the specified decimate limit and the required point should be revealed.

In [ ]:
%%opts Points (color="cyan") [tools=["hover"] width=900 height=650] 
datashade(r_bundled, width=900, height=650) * \
decimate( hv.Points(r_nodes_named),max_samples=2000)

If you click around and hover, you should see interesting groups of nodes, and can then set up further interactive tools using [HoloViews' stream support](http://holoviews.org/user_guide/Responding_to_Events.html) to reveal aspects relevant to your research interests or questions.

As you can see, datashader lets you work with very large graph datasets, though there are a number of decisions to make by trial and error, you do have to be careful when doing computationally expensive operations like edge bundling, and interactive information will only be available for a limited subset of the data at any one time due to data-size limitations of current web browsers.